In [26]:
import pandas as pd
import numpy as np
from typing import Optional, List, Dict
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

# Import the new Pydantic models from running_results
import importlib
import running_results.models
importlib.reload(running_results.models)

from running_results.models import (
    NormalizedRaceResult,
    ColumnMapping,
    TimeParser,
    RaceResultsNormalizer,
    RaceCategory,
    Gender,
    RaceStatus,
    normalize_race_results,
    fix_malformed_time,
    normalize_club_name,
    parse_age_category,
)


In [31]:
# Correct mapping for Tinto CSVs
tinto_mapping = ColumnMapping(
    position_overall='RunnerPosition',
    name='Name',  # Combined from Surname and Firstname
    club='Club',
    finish_time_minutes='FinishTime',  # Map to minutes field - times are in MM:SS format
    age_category='RunnerCategory'
)

# Create a time parser for MM:SS format
time_parser = TimeParser(format='MM:SS')


In [32]:
# Check if normalized data already exists
normalized_dir = Path('../tinto_normalized')
normalized_file = normalized_dir / 'tinto_all_years.parquet'

if normalized_file.exists():
    print("Loading pre-normalized data from disk...")
    all_data = pd.read_parquet(normalized_file)
    print(f"Loaded {len(all_data)} results from {all_data['race_year'].min()}-{all_data['race_year'].max()}")
else:
    print("Normalizing data from CSV files...")
    years = {}
    for year in range(1985, 2004):
        try:
            # Load Tinto data
            print(f"  Loading Tinto {year} data...")
            df = pd.read_csv(f'../tinto/{year}.csv')
            
            # Combine Surname and Firstname into a single Name column
            if 'Surname' in df.columns and 'Firstname' in df.columns:
                df['Name'] = df['Firstname'].fillna('') + ' ' + df['Surname'].fillna('')
                df['Name'] = df['Name'].str.strip()

            # Create normalizer for Tinto
            tinto_normalizer = RaceResultsNormalizer(
                mapping=tinto_mapping,
                time_parser=time_parser,
                race_name=f'Tinto {year}',
                race_year=year,
                race_category=RaceCategory.FELL_RACE
            )

            # Normalize the data
            tinto_normalized = tinto_normalizer.normalize(
                df, 
                return_dataframe=True
            )

            years[year] = tinto_normalized
            print(f"    Loaded {len(tinto_normalized)} results")
        except Exception as e:
            print(f"    ERROR for year {year}: {e}")
            import traceback
            traceback.print_exc()
            raise
    
    # Combine all years
    all_data = pd.concat(years.values(), ignore_index=True)
    
    # Save normalized data
    print(f"\nSaving normalized data to {normalized_file}...")
    normalized_dir.mkdir(exist_ok=True)
    all_data.to_parquet(normalized_file, index=False)
    
    # Also save as CSV for easier inspection
    csv_file = normalized_dir / 'tinto_all_years.csv'
    all_data.to_csv(csv_file, index=False)
    print(f"Saved {len(all_data)} results")

print(f"\nData summary:")
print(f"  Total results: {len(all_data)}")
print(f"  Years: {all_data['race_year'].min()}-{all_data['race_year'].max()}")
print(f"  Results with finish times: {all_data['finish_time_seconds'].notna().sum()}")
print(f"  Unique clubs: {all_data['club'].nunique()}")
print(f"  Unique runners: {all_data['name'].nunique()}")


Normalizing data from CSV files...
  Loading Tinto 1985 data...
    Loaded 65 results
  Loading Tinto 1986 data...
    Loaded 112 results
  Loading Tinto 1987 data...
    Loaded 141 results
  Loading Tinto 1988 data...
    Loaded 141 results
  Loading Tinto 1989 data...
    Loaded 145 results
  Loading Tinto 1990 data...
    Loaded 153 results
  Loading Tinto 1991 data...
    Loaded 166 results
  Loading Tinto 1992 data...
    Loaded 158 results
  Loading Tinto 1993 data...
    Loaded 151 results
  Loading Tinto 1994 data...
    Loaded 135 results
  Loading Tinto 1995 data...
    Loaded 102 results
  Loading Tinto 1996 data...
    Loaded 149 results
  Loading Tinto 1997 data...
    Loaded 99 results
  Loading Tinto 1998 data...
    Loaded 116 results
  Loading Tinto 1999 data...
    Loaded 147 results
  Loading Tinto 2000 data...
    Loaded 128 results
  Loading Tinto 2001 data...
    Loaded 121 results
  Loading Tinto 2002 data...
    Loaded 171 results
  Loading Tinto 2003 data...
  

In [4]:
import matplotlib.pyplot as plt

In [33]:
# Show examples of the normalizations applied
print("=" * 60)
print("NORMALIZATION EXAMPLES")
print("=" * 60)

# Show club name normalization
print("\n1. CLUB NAME NORMALIZATION:")
sample_clubs = all_data[all_data['club'].notna()]['club'].value_counts().head(10)
print(sample_clubs)

# Show age category parsing
print("\n2. AGE CATEGORY PARSING:")
sample_cats = all_data[all_data['age_category'].notna()]['age_category'].value_counts().head(10)
print(sample_cats)

# Show gender extraction from age categories
print("\n3. GENDER EXTRACTION (from age categories):")
gender_counts = all_data['gender'].value_counts()
print(gender_counts)

# Show sample records
print("\n4. SAMPLE NORMALIZED RECORDS:")
sample = all_data[all_data['age_category'].notna()].head(5)
print(sample[['name', 'club', 'age_category', 'gender', 'finish_time_minutes', 'race_year']])


NORMALIZATION EXAMPLES

1. CLUB NAME NORMALIZATION:
club
Carnethy       463
Westerlands    149
Livingston     140
Fife            80
Lomond          69
Ochil           67
Lochaber        61
Solway          60
HBT             57
HELP            53
Name: count, dtype: int64

2. AGE CATEGORY PARSING:
age_category
M40      578
F        230
M50      218
S         88
M         73
U20       64
F40       62
Vet       43
M60       33
S/Vet     20
Name: count, dtype: int64

3. GENDER EXTRACTION (from age categories):
gender
M    843
F    298
Name: count, dtype: int64

4. SAMPLE NORMALIZED RECORDS:
          name      club age_category gender  finish_time_minutes  race_year
4    Ross Hope  Carnethy          U20   None            32.383333       1985
13  Bill Gauld  Carnethy          M40      M            35.683333       1985
14   K. Duncan    Dollar          U20   None            35.966667       1985
21   N. Slorch    Dollar          U20   None            36.700000       1985
24  Pete Duffy  Aber

In [11]:
# Check what columns are actually in the 1985 CSV
df_sample = pd.read_csv('../tinto/1985.csv')
print("Actual columns in CSV:")
print(df_sample.columns.tolist())
print("\nFirst few rows:")
print(df_sample.head())


Actual columns in CSV:
['RunnerPosition', 'Surname', 'Firstname', 'Club', 'RunnerCategory', 'FinishTime']

First few rows:
   RunnerPosition     Surname Firstname      Club RunnerCategory FinishTime
0               1  Farningham      Alan      Fife            NaN      31:33
1               2   McGonigle    Dermot    Dundee            NaN      31:46
2               3   Spenceley      Andy  Carnethy            NaN      32:05
3               4      Burton      Mike  Carnethy            NaN      32:22
4               5        Hope      Ross  Carnethy              J      32:23


In [ ]:
# Demo: New Database & Import Features

Now let's demonstrate the new database and import capabilities:
- Import results into a database
- Query across multiple years
- Add new results from URLs or files

Columns: ['RunnerPosition', 'Surname', 'Firstname', 'Club', 'RunnerCategory', 'FinishTime']
Shape: (128, 6)

First few rows:
  RunnerPosition Surname Firstname         Club RunnerCategory FinishTime
0              1   Rigby      Mark  Westerlands            NaN      32:23
1              2  Lenton       Tim      Lothian            NaN      32:40
2              3  Connor      Brad     Carnethy            NaN      33:11
3              4   Coyle      John       Solway            NaN      33:26
4              5     Lee       Rob      Lothian            NaN      33:37

Success! Loaded 128 results
   position_overall position_gender position_category         name bib_number  \
0               1.0            None              None   Mark Rigby       None   
1               2.0            None              None   Tim Lenton       None   
2               3.0            None              None  Brad Connor       None   
3               4.0            None              None   John Coyle       None 

In [34]:
# Update __init__.py to export new classes first
init_file = '../running_results/__init__.py'
with open(init_file, 'r') as f:
    content = f.read()

# Add imports if not already there
if 'from .database import' not in content:
    lines = content.split('\n')
    # Find where to insert (after other imports)
    insert_idx = None
    for i, line in enumerate(lines):
        if line.startswith('from .models import'):
            insert_idx = i
            break
    
    if insert_idx:
        # Insert new imports
        new_imports = [
            'from .database import RaceResultsDatabase',
            'from .importers import ResultsImporter, SmartImporter',
            'from .manager import RaceResultsManager',
            ''
        ]
        for j, imp in enumerate(new_imports):
            lines.insert(insert_idx + j, imp)
        
        # Add to __all__
        all_idx = None
        for i, line in enumerate(lines):
            if line.strip().startswith("'parse_age_category',"):
                all_idx = i
                break
        
        if all_idx:
            new_exports = [
                "    'RaceResultsDatabase',",
                "    'ResultsImporter',",
                "    'SmartImporter',",
                "    'RaceResultsManager',"
            ]
            for j, exp in enumerate(new_exports):
                lines.insert(all_idx + 1 + j, exp)
        
        # Write back
        with open(init_file, 'w') as f:
            f.write('\n'.join(lines))
        
        print("✓ Updated __init__.py")
else:
    print("✓ Already updated")


✓ Updated __init__.py


In [35]:
# Reload the package to get new classes
import importlib
import running_results
importlib.reload(running_results)

from running_results import RaceResultsManager

print("✓ Imported RaceResultsManager")

✓ Imported RaceResultsManager


In [37]:
# Initialize the race results manager with a database
manager = RaceResultsManager('../tinto_results.db')

# Import all the Tinto data we already normalized
print("Importing Tinto results into database...")

def to_none_if_nan(val):
    """Convert NaN to None for Pydantic"""
    if pd.isna(val):
        return None
    return val

for year in range(1985, 2004):
    year_data = all_data[all_data['race_year'] == year]
    
    # Convert DataFrame to list of NormalizedRaceResult objects
    results = []
    for _, row in year_data.iterrows():
        result = NormalizedRaceResult(
            position_overall=to_none_if_nan(row['position_overall']),
            name=to_none_if_nan(row['name']),
            club=to_none_if_nan(row['club']),
            age_category=to_none_if_nan(row['age_category']),
            gender=to_none_if_nan(row['gender']),
            finish_time_seconds=to_none_if_nan(row['finish_time_seconds']),
            finish_time_minutes=to_none_if_nan(row['finish_time_minutes']),
            race_status=to_none_if_nan(row['race_status'])
        )
        results.append(result)
    
    # Add to database
    count = manager.add_results(
        results,
        race_name='Tinto',
        race_year=year,
        race_category='fell_race'
    )
    print(f"  {year}: {count} results")

print("\n✓ Database populated!")

Importing Tinto results into database...
  1985: 65 results
  1986: 112 results
  1987: 141 results
  1988: 141 results
  1989: 145 results
  1990: 153 results
  1991: 166 results
  1992: 158 results
  1993: 151 results
  1994: 135 results
  1995: 102 results
  1996: 149 results
  1997: 99 results
  1998: 116 results
  1999: 147 results
  2000: 128 results
  2001: 121 results
  2002: 171 results
  2003: 153 results

✓ Database populated!


In [38]:
# Now demonstrate querying the database

print("=" * 60)
print("DATABASE QUERY EXAMPLES")
print("=" * 60)

# List all races
print("\n1. ALL RACES IN DATABASE:")
races = manager.list_races()
print(races)

# Get all Tinto results across all years
print("\n2. TINTO RESULTS (all years):")
tinto_all = manager.get_race('Tinto')
print(f"Total results: {len(tinto_all)}")
print(f"Years: {tinto_all['race_year'].min()} - {tinto_all['race_year'].max()}")
print(tinto_all.head())

# Get specific year
print("\n3. TINTO 2000 RESULTS:")
tinto_2000 = manager.get_race('Tinto', year=2000)
print(f"Results: {len(tinto_2000)}")
print(tinto_2000[['name', 'club', 'finish_time_minutes', 'position_overall']].head())

# Get a runner's history
print("\n4. RUNNER HISTORY (Alan Farningham):")
history = manager.get_runner_history('Farningham')
print(history[['race_year', 'position_overall', 'finish_time_minutes', 'club']])

# Search by club
print("\n5. CARNETHY RESULTS (all years):")
carnethy = manager.search_results(club='Carnethy')
print(f"Total Carnethy results: {len(carnethy)}")
print(f"Unique runners: {carnethy['name'].nunique()}")
print(carnethy.groupby('race_year').size())

DATABASE QUERY EXAMPLES

1. ALL RACES IN DATABASE:
  race_name race_category  num_years  first_year  last_year  total_results
0     Tinto     fell_race         19        1985       2003           2553

2. TINTO RESULTS (all years):
Total results: 2553
Years: 1985 - 2003
  race_name  race_year race_date source_url  position_overall position_gender  \
0     Tinto       1985      None       None               1.0            None   
1     Tinto       1985      None       None               2.0            None   
2     Tinto       1985      None       None               3.0            None   
3     Tinto       1985      None       None               4.0            None   
4     Tinto       1985      None       None               5.0            None   

  position_category              name bib_number gender age_category  \
0              None   Alan Farningham       None   None         None   
1              None  Dermot McGonigle       None   None         None   
2              None    And

## Adding New Results

The manager makes it easy to add new results from files or URLs:

In [39]:
# Example: Add results from a file
# (This demonstrates how you would add a new year)

# Simulated example - in practice you would do:
"""
# Add from CSV file:
count = manager.add_from_file(
    'tinto/2013.csv',
    race_name='Tinto',
    race_year=2013,
    race_category='fell_race'
)
print(f"Added {count} results from file")

# Add from URL:
count = manager.add_from_url(
    'https://example.com/tinto-2024-results',
    race_name='Tinto',
    race_year=2024,
    race_category='fell_race'
)
print(f"Added {count} results from URL")

# The manager handles:
# - Fetching the data (from file or web)
# - Auto-detecting columns
# - Normalizing the data
# - Storing in the database
# All in one simple call!
"""

print("Example code shown above - demonstrates:")
print("  ✓ Adding from files with add_from_file()")
print("  ✓ Adding from URLs with add_from_url()")
print("  ✓ Auto-detection of columns")
print("  ✓ Automatic normalization")
print("  ✓ Database storage")

Example code shown above - demonstrates:
  ✓ Adding from files with add_from_file()
  ✓ Adding from URLs with add_from_url()
  ✓ Auto-detection of columns
  ✓ Automatic normalization
  ✓ Database storage


## Summary

The running_results package now includes:

1. **Data Normalization** (models.py)
   - Pydantic validation with 23 optional fields
   - Auto-correction of malformed times (::, :40:56)
   - Club name standardization (removes suffixes, handles U/A)
   - Age category parsing (V→M40, SV→M50, FV→F40, etc.)
   - DNF/DNS/DSQ status handling
   
2. **Database Storage** (database.py)
   - SQLite-based persistent storage
   - Track races across multiple years
   - Query by race, runner, club, year, etc.
   - Indexed for fast searches
   
3. **Data Import** (importers.py)
   - Web scraping from URLs (HTML tables)
   - File import (CSV, TSV, Excel, HTML)
   - Auto-detection of formats and columns
   - Text import with delimiter detection
   
4. **Unified Manager** (manager.py)
   - Simple API for all operations
   - `add_from_url()` / `add_from_file()` / `add_results()`
   - `get_race()` / `get_runner_history()` / `search_results()`
   - Automatic normalization on import
   
The Tinto database now contains **2,553 normalized results** spanning **19 years** (1985-2003)!